In [1]:
# Imports
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# PCA-feature selection
from sklearn.decomposition import PCA

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

# For evaluation
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import time

In [2]:
telcom = pd.read_csv("clean.csv")

In [3]:
telcom.head()

customerID  gender SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female            No     Yes         No       1           No   
1  5575-GNVDE    Male            No      No         No      34          Yes   
2  3668-QPYBK    Male            No      No         No       2          Yes   
3  7795-CFOCW    Male            No      No         No      45           No   
4  9237-HQITU  Female            No      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... TechSupport  \
0  No phone service             DSL             No  ...          No   
1                No             DSL            Yes  ...          No   
2                No             DSL            Yes  ...          No   
3  No phone service             DSL            Yes  ...         Yes   
4                No     Fiber optic             No  ...          No   

  StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No              No  Month-to-month              Yes   
1          No              No        One year               No   
2          No              No  Month-to-month              Yes   
3          No              No        One year               No   
4          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges TotalCharges  Churn  tenure_group  
0           Electronic check          29.85        29.85     No   Tenure_0-12  
1               Mailed check          56.95      1889.50     No  Tenure_24-48  
2               Mailed check          53.85       108.15    Yes   Tenure_0-12  
3  Bank transfer (automatic)          42.30      1840.75     No  Tenure_24-48  
4           Electronic check          70.70       151.65    Yes   Tenure_0-12  

[5 rows x 22 columns]

In [4]:
#Separating catagorical and numerical columns
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

In [5]:
print(len(cat_cols),"//",len(num_cols))

17 // 3


In [6]:
# Dropping customerID column
telcom.drop(columns=Id_col, axis=1, inplace=True)

In [7]:
# Encoding categorical varaible
telcom["gender"] = telcom["gender"].apply(lambda x: 1 if x=="Male" else 0)

In [8]:
# Columns with only two values: "Yes" and "No"
binary_cols = telcom.nunique()[telcom.nunique() < 3].keys().tolist()
binary_cols.remove("gender")

for x in binary_cols:
    telcom[x] = telcom[x].apply(lambda x: 1 if x=="Yes" else 0)

In [9]:
cat_cols = [x for x in cat_cols if x not in binary_cols]
cat_cols.remove("gender")

In [10]:
cat_cols

['MultipleLines',
 'InternetService',
 'Contract',
 'PaymentMethod',
 'tenure_group']

In [11]:
for x in cat_cols:
    print(telcom[x].value_counts())

No                  3385
Yes                 2967
No phone service     680
Name: MultipleLines, dtype: int64
Fiber optic    3096
DSL            2416
No             1520
Name: InternetService, dtype: int64
Month-to-month    3875
Two year          1685
One year          1472
Name: Contract, dtype: int64
Electronic check             2365
Mailed check                 1604
Bank transfer (automatic)    1542
Credit card (automatic)      1521
Name: PaymentMethod, dtype: int64
Tenure_0-12     2175
Tenure_24-48    1594
Tenure_gt_60    1407
Tenure_12-24    1024
Tenure_48-60     832
Name: tenure_group, dtype: int64


In [12]:
telcom["MultipleLines"] = telcom.MultipleLines.apply(lambda x: "MultipleLines_"+x)

In [13]:
# Function for encoding leftover categorical variables.
def convertor(x,telcom,df):
    enc = OneHotEncoder(drop='first', sparse=False)
    cnn = enc.fit_transform(telcom[x].values.reshape(-1,1))
    for y,i in zip(telcom[x].unique()[1:],range(5)):
        df[y] = pd.Series(data=cnn[:,i])
    

In [14]:
df_enc = pd.DataFrame()

for x in cat_cols:
    convertor(x,telcom,df_enc)

In [15]:
df_enc.head()

MultipleLines_No  MultipleLines_Yes  Fiber optic   No  One year  Two year  \
0               1.0                0.0          0.0  0.0       0.0       0.0   
1               0.0                0.0          0.0  0.0       1.0       0.0   
2               0.0                0.0          0.0  0.0       0.0       0.0   
3               1.0                0.0          0.0  0.0       1.0       0.0   
4               0.0                0.0          1.0  0.0       0.0       0.0   

   Mailed check  Bank transfer (automatic)  Credit card (automatic)  \
0           0.0                        1.0                      0.0   
1           0.0                        0.0                      1.0   
2           0.0                        0.0                      1.0   
3           0.0                        0.0                      0.0   
4           0.0                        1.0                      0.0   

   Tenure_24-48  Tenure_12-24  Tenure_gt_60  Tenure_48-60  
0           0.0           0.0           0.0           0.0  
1           0.0           1.0           0.0           0.0  
2           0.0           0.0           0.0           0.0  
3           0.0           1.0           0.0           0.0  
4           0.0           0.0           0.0           0.0

In [16]:
df_enc.shape

(7032, 13)

In [17]:
telcom.shape

(7032, 21)

In [18]:
# Dropping cat_cols from telcom and concating encoded columns
telcom = telcom.drop(columns=cat_cols, axis=1)
telcom = pd.concat([telcom,df_enc],axis=1)
telcom.head()

gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0       0              0        1           0       1             0   
1       1              0        0           0      34             1   
2       1              0        0           0       2             1   
3       1              0        0           0      45             0   
4       0              0        0           0       2             1   

   OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  ...   No  \
0               0             1                 0            0  ...  0.0   
1               1             0                 1            0  ...  0.0   
2               1             1                 0            0  ...  0.0   
3               1             0                 1            1  ...  0.0   
4               0             0                 0            0  ...  0.0   

   One year  Two year  Mailed check  Bank transfer (automatic)  \
0       0.0       0.0           0.0                        1.0   
1       1.0       0.0           0.0                        0.0   
2       0.0       0.0           0.0                        0.0   
3       1.0       0.0           0.0                        0.0   
4       0.0       0.0           0.0                        1.0   

   Credit card (automatic)  Tenure_24-48  Tenure_12-24  Tenure_gt_60  \
0                      0.0           0.0           0.0           0.0   
1                      1.0           0.0           1.0           0.0   
2                      1.0           0.0           0.0           0.0   
3                      0.0           0.0           1.0           0.0   
4                      0.0           0.0           0.0           0.0   

   Tenure_48-60  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  

[5 rows x 29 columns]

In [19]:
# Separating target variable
y = telcom.pop("Churn")
X = telcom

In [20]:
# Scaling tenure column
scale = StandardScaler()
X["tenure"] = scale.fit_transform(X["tenure"].values.reshape(-1,1))
X.head()

gender  SeniorCitizen  Partner  Dependents    tenure  PhoneService  \
0       0              0        1           0 -1.280248             0   
1       1              0        0           0  0.064303             1   
2       1              0        0           0 -1.239504             1   
3       1              0        0           0  0.512486             0   
4       0              0        0           0 -1.239504             1   

   OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport  ...   No  \
0               0             1                 0            0  ...  0.0   
1               1             0                 1            0  ...  0.0   
2               1             1                 0            0  ...  0.0   
3               1             0                 1            1  ...  0.0   
4               0             0                 0            0  ...  0.0   

   One year  Two year  Mailed check  Bank transfer (automatic)  \
0       0.0       0.0           0.0                        1.0   
1       1.0       0.0           0.0                        0.0   
2       0.0       0.0           0.0                        0.0   
3       1.0       0.0           0.0                        0.0   
4       0.0       0.0           0.0                        1.0   

   Credit card (automatic)  Tenure_24-48  Tenure_12-24  Tenure_gt_60  \
0                      0.0           0.0           0.0           0.0   
1                      1.0           0.0           1.0           0.0   
2                      1.0           0.0           0.0           0.0   
3                      0.0           0.0           1.0           0.0   
4                      0.0           0.0           0.0           0.0   

   Tenure_48-60  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  

[5 rows x 28 columns]

# Feature Selection

In [21]:
pca = PCA(n_components = None)
pca.fit(X)

PCA()

In [22]:
# printing the explained variances
print("Variances (Percentage):")
print(pca.explained_variance_ratio_ * 100)
print()

Variances (Percentage):
[9.99897668e+01 1.01582989e-02 7.07731979e-06 6.44329532e-06
 5.42921973e-06 4.96077114e-06 4.87152911e-06 4.49571569e-06
 3.95199039e-06 3.82589279e-06 3.50276643e-06 3.06920007e-06
 2.95664719e-06 2.89026711e-06 2.80620793e-06 2.62449798e-06
 2.44100772e-06 2.42790312e-06 2.14707582e-06 2.14247527e-06
 2.02890369e-06 1.87135443e-06 1.10365751e-06 1.01512874e-06
 7.89043724e-07 4.72446381e-08 1.14341186e-08 9.36972581e-31]



In [23]:
print("Cumulative Variances (Percentage):")
print(pca.explained_variance_ratio_.cumsum() * 100)
print()

Cumulative Variances (Percentage):
[ 99.98976677  99.99992507  99.99993215  99.99993859  99.99994402
  99.99994898  99.99995385  99.99995835  99.9999623   99.99996613
  99.99996963  99.9999727   99.99997565  99.99997854  99.99998135
  99.99998397  99.99998642  99.99998884  99.99999099  99.99999313
  99.99999516  99.99999703  99.99999814  99.99999915  99.99999994
  99.99999999 100.         100.        ]



In [24]:
# First component alone gives 99.98%

In [25]:
pca = PCA(n_components = 0.85)
pca.fit(X)
print("Cumulative Variances (Percentage):")
print(np.cumsum(pca.explained_variance_ratio_ * 100))
components = len(pca.explained_variance_ratio_)
print(f'Number of components: {components}')


Cumulative Variances (Percentage):
[99.98976677]
Number of components: 1


In [26]:
pca_components = abs(pca.components_)
print(pca_components)

[[1.02026878e-08 1.66662602e-05 7.03368736e-05 1.30485215e-05
  3.64339757e-04 1.47375312e-05 8.23076035e-05 1.06969329e-04
  1.09575080e-04 8.66648058e-05 1.10682259e-04 1.11787473e-04
  3.42173190e-05 8.64188274e-03 9.99962534e-01 1.47375312e-05
  1.02199471e-04 7.90244041e-05 6.80869317e-05 3.06118197e-05
  6.74165826e-05 3.31768475e-05 1.25893803e-05 5.45599355e-05
  3.27928657e-05 4.72764669e-06 3.60349281e-05 1.12831717e-04]]


In [27]:
print('Top 6 most important features')
print('===============================================')
for row in range(pca_components.shape[0]):
    temp = np.argpartition(-(pca_components[row]), 6)
    
    indices = temp[np.argsort((-pca_components[row])[temp])][:6]
    
    print(f'Component {row}: {X.columns[indices].to_list()}')

Top 6 most important features
Component 0: ['TotalCharges', 'MonthlyCharges', 'tenure', 'Tenure_48-60', 'StreamingMovies', 'StreamingTV']


In [28]:
Selected = ['TotalCharges', 'MonthlyCharges', 'tenure', 'Tenure_48-60', 'StreamingMovies', 'StreamingTV']

In [29]:
X = X[Selected]

In [30]:
# Splitting into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, stratify=y)
print("X_train, X_test shapes: ",X_train.shape,X_test.shape,"\n","y_train, y_test shapes: ",y_train.shape,y_test.shape)

X_train, X_test shapes:  (6328, 6) (704, 6) 
 y_train, y_test shapes:  (6328,) (704,)


# BaseLine Modelling with selected features

In [31]:
# Classifiers
names = [
    "Logistic Regression",
    "KNN Classifier",
    "Decision Tree",
    "Random Forest",
    "XGBoost",
    "AdaBoost",
]

models = [
    LogisticRegression(random_state=42),
    KNeighborsClassifier(n_neighbors = 5, n_jobs = -1),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(n_estimators = 100),
    XGBClassifier(),
    AdaBoostClassifier()
]

In [32]:
# Function to return summary of baseline models
def score(X_train, y_train, X_val, y_val, names = names, models = models):
    score_df, score_train, score_val = pd.DataFrame(), [], []
    x = time.time()
    for model in models:
        model.fit(X_train, y_train)
        y_train_pred, y_val_pred = model.predict(X_train), model.predict(X_val)
        score_train.append(accuracy_score(y_train, y_train_pred))
        score_val.append(accuracy_score(y_val, y_val_pred))
    
    score_df["Classifier"], score_df["Training accuracy"], score_df["Test accuracy"] = names, score_train, score_val
    score_df.sort_values(by = 'Test accuracy', ascending = False, inplace = True)
    return score_df

In [33]:
# Summary of baseline models
score(X_train, y_train, X_test, y_test, names = names, models = models)

Classifier  Training accuracy  Test accuracy
5             AdaBoost           0.794090       0.792614
4              XGBoost           0.882901       0.786932
0  Logistic Regression           0.786662       0.776989
1       KNN Classifier           0.828856       0.764205
3        Random Forest           0.989096       0.761364
2        Decision Tree           0.989254       0.720170

In [34]:
# AdaBoost gives the best train as well as test accuracy score,
# thus going ahead I will suggest the use of AdaBoost Model

In [35]:
abd = AdaBoostClassifier()

In [36]:
abd.fit(X_train, y_train)
y_train_pred, y_test_pred = abd.predict(X_train), abd.predict(X_test)

In [37]:
confusion_matrix(y_test,y_test_pred)

array([[473,  44],
       [102,  85]], dtype=int64)